In [ ]:
import cv2
import os
import numpy as np
from ultralytics import YOLO

In [ ]:
def draw_box(img, xyxy, conf, cls_name):
    x1, y1, x2, y2 = map(int, xyxy)
    label = '{:.2f}'.format(conf)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 200, 0), 2)
    (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(img, (x1, y1 - h - 6), (x1 + w, y1), (0, 200, 0), -1)
    cv2.putText(img, label, (x1, y1 - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

In [34]:
def process_image(model, img_path, out_dir=r'../result/', conf_threshold=0.25):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(img_path)
    results = model(img, conf=conf_threshold, imgsz=640)
    r = results[0]
    names = model.names
    car_class_ids = [k for k, v in names.items() if v.lower() == 'car']
    car_ids_set = set(car_class_ids)

    for box in r.boxes:
        cls_id = int(box.cls.cpu().numpy()) if hasattr(box, 'cls') else int(box.cls)
        conf = float(box.conf.cpu().numpy()) if hasattr(box, 'conf') else float(box.conf)
        xyxy = box.xyxy.cpu().numpy().reshape(-1) if hasattr(box, 'xyxy') else box.xyxy
        if cls_id in car_ids_set:
            cls_name = names[cls_id]
            draw_box(img, xyxy, conf, cls_name)

    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, os.path.basename(img_path))
    cv2.imwrite(out_path, img)
    print('Saved:', out_path)
    return out_path

In [ ]:
def main():
    yolo_versions = ('n', 's', 'm', 'l', 'x')
    version = yolo_versions[2]
    image_path = r""

    model = YOLO('yolo11{}.pt'.format(version))
    process_image(model, image_path)

In [35]:
main()


0: 384x640 42 cars, 346.3ms
Speed: 2.9ms preprocess, 346.3ms inference, 7.6ms postprocess per image at shape (1, 3, 384, 640)
Saved: ../result/1.jpg


C:\Users\da1zy\AppData\Local\Temp\ipykernel_13400\2780654300.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cls_id = int(box.cls.cpu().numpy()) if hasattr(box, 'cls') else int(box.cls)
C:\Users\da1zy\AppData\Local\Temp\ipykernel_13400\2780654300.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf = float(box.conf.cpu().numpy()) if hasattr(box, 'conf') else float(box.conf)
